In [ ]:
import pandas as pd
import json

df = pd.read_csv("nikkei.csv", encoding="cp932")

# 直近データだけ使う（例：最後30日）
context_length = 30
recent = df.tail(context_length)

predict_data = {
    "start": recent["date"].iloc[0],
    "target": recent["close"].tolist()
}

with open("predict.json", "w", encoding="utf-8") as f:
    json.dump(predict_data, f)

import sagemaker

session = sagemaker.Session()
bucket = session.default_bucket()
prefix = "nikkei-deepar"

s3_predict_path = session.upload_data(
    path="predict.json",
    bucket=bucket,
    key_prefix=f"{prefix}/predict"
)
print(s3_predict_path)


from sagemaker.estimator import Estimator

training_job_name = "forecasting-deepar-2026-01-10-02-08-21-129"

estimator = Estimator.attach(training_job_name)

transformer = estimator.transformer(
    instance_count=1,
    instance_type="ml.m5.large",
    output_path=f"s3://{bucket}/{prefix}/output",
    assemble_with="Line"
)

transformer.transform(
    data=s3_predict_path,
    content_type="application/jsonlines",
    split_type="Line"
)

transformer.wait()